# Explore IBL data Pipeline

Here we introduce some useful DataJoint tools to explore the IBL data pipeline

First thing first, let's **import DataJoint and the IBL pipeline package**.

In [ ]:
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior, ephys
from ibl_pipeline.analyses import behavior as behavior_analyses

# 1. Browse schemas you have access to - `dj.list_schemas()`

In [ ]:
dj.list_schemas()


## Major schemas:   

Meta data from **Alyx**: `ibl_reference`, `ibl_subject`, `ibl_action`, `ibl_acquisition`, and `ibl_data`  
Imported data from **FlatIron**: `ibl_behavior` and `ibl_ephys`  
Computed analzyed results: `ibl_analyses_behavior`  

# 2. Browse tables in a schema - `dj.Diagram`

**Table tiers**:  
Manual table: green box  
Lookup table: gray box  
Imported table: blue oval  
Computed table: red circle  
Part table: plain text

**Dependencies**:  
One-to-one primary: thick solid line  
One-to-many primary: thin solid line  
Secondary foreign key reference: dashed line
Renamed secondary foreign key references: orange dot

## Show tables in the whole schema

In [ ]:
dj.Diagram(reference)

## Show diagram of arbitruary parts of the database

In [ ]:
dj.Diagram(subject)

A combination of arbitruary tables:

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(subject.Line) + dj.Diagram(subject.SubjectLab) + dj.Diagram(subject.SubjectProject)

The tables could be from different schemas:

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(acquisition.Session)

# 3. Getting the detailed definition of a table - `table.describe()`

In [ ]:
subject.Subject.describe();

# 4. Browsing of data - queries

Query all subjects

In [ ]:
subject.Subject()

## Restriction  `&`: filtering data

### Restriction: Query one subject

In [ ]:
# restrict by string
subject.Subject & 'subject_nickname="ibl_witten_10"'

In [ ]:
# restrict by dictionary
from uuid import UUID
subject.Subject & {'subject_uuid': UUID('00c60db3-74c3-4ee2-9df9-2c84acf84e92')}

In [ ]:
subject.Subject & {'sex': 'm'}

### Restriction: Query subjects born after a date

In [ ]:
subject.Subject & 'subject_birth_date > "2019-01-01"'

### Restriction: subjects within a range of dates

In [ ]:
subject.Subject & 'subject_birth_date between "2019-01-01" and "2019-04-01"'

### Restriction: Query subjects on multiple attributes

In [ ]:
subject.Subject & 'subject_birth_date > "2019-01-01"' & 'sex="M"'

### Restriction: Query subjects restricted by other tables

In [ ]:
# subjects with Ephys recording
subject.Subject & ephys.Ephys

In [ ]:
# subjects without Ephys recording
subject.Subject - ephys.Ephys

## Join `*`:  gather information from different tables

In [ ]:
subject.Subject * acquisition.Session

## Projection `.proj()`: focus on attributes of interest

In [ ]:
subject.Subject.proj()

In [ ]:
subject.Subject.proj('subject_birth_date', 'sex')

### rename attribute with ***proj()***

In [ ]:
subject.Subject.proj('sex', dob='subject_birth_date')

### perform simple computations with ***proj***

**Example 1: Get date of a session:**

In [ ]:
sessions_with_date = acquisition.Session.proj(session_date='date(session_start_time)')

In [ ]:
sessions_with_date

**Example 2: Age of the animal when performing each session?**

In [ ]:
# First get the date of birth and the session date into the same query
q = subject.Subject * acquisition.Session

In [ ]:
q

In [ ]:
# Then compute the age
q_with_age = q.proj(age='datediff(session_start_time, subject_birth_date)')
q_with_age

## Aggregation `.aggr()`: simple computation of one table against another table

Example: how many sessions does each subject do so far?

In [ ]:
subject.Subject.aggr(acquisition.Session, 'subject_nickname', n='count(*)')

# 5. Fetching data

## Fetch all fields: `fetch()`

In [ ]:
# fetch all data from a table
subjs = subject.Subject.fetch()
subjs

In [ ]:
subjs['subject_uuid']

In [ ]:
subjs['subject_birth_date']

In [ ]:
# fetch as a list of dictionaries
subjs_dict = subject.Subject.fetch(as_dict=True)
subjs_dict

In [ ]:
# fetch as pandas dataframe
subjs_df = subject.Subject.fetch(format='frame')
subjs_df

In [ ]:
# fetch the primary key
pk = subject.Subject.fetch('KEY')
pk

In [ ]:
# fetch specific attributes
dob, sex = subject.Subject.fetch('subject_birth_date', 'sex')

In [ ]:
dob

In [ ]:
info = subject.Subject.fetch('subject_birth_date', 'sex', as_dict=True)
info

## fetch data only from one entry: `fetch1`

In [ ]:
ibl_witten_10 = (subject.Subject & {'subject_nickname': 'ibl_witten_10'}).fetch1('KEY')  # "fetch1()" because we know there's only one

In [ ]:
ibl_witten_10

In [ ]:
IBL_10 = (subject.Subject & {'subject_nickname': 'IBL_10'}).fetch1()

In [ ]:
IBL_10